<a href="https://colab.research.google.com/github/mohamed34318/Movie-Recommendation-System-Collaborative-filtering-/blob/master/Movie_Recommendation_Engine_(Collaborative_Filtering).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# natural language toolkit
import re
import nltk
pd.set_option('display.max_columns', None)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [85]:
df = pd.read_csv('/content/data/dd/IMDB_Top250Engmovies2_OMDB_Detailed.csv', index_col=False)
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)
df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              250 non-null    object 
 1   Year               250 non-null    int64  
 2   Rated              250 non-null    object 
 3   Released           248 non-null    object 
 4   Runtime            250 non-null    object 
 5   Genre              250 non-null    object 
 6   Director           250 non-null    object 
 7   Writer             249 non-null    object 
 8   Actors             250 non-null    object 
 9   Plot               250 non-null    object 
 10  Language           250 non-null    object 
 11  Country            250 non-null    object 
 12  Awards             245 non-null    object 
 13  Poster             250 non-null    object 
 14  Ratings.Source     250 non-null    object 
 15  Ratings.Value      250 non-null    object 
 16  Metascore          177 non

In [87]:
df.shape

(250, 37)

#data Preprocessing

In [88]:
# we will convert Plot description of into list of words
df['clean_plot'] = df['Plot'].str.lower() # convert to lower
df['clean_plot'] = df['clean_plot'].apply(lambda x:re.sub('[^a-zA-Z]' , ' ', x)) # remove any symboles
df['clean_plot'] = df['clean_plot'].apply(lambda x:re.sub('\s+' , ' ', x)) # remove additional Space
df['clean_plot']

# after that will tokanize sentance into list of words + remove Stop words

0      two imprisoned men bond over a number of years...
1      the aging patriarch of an organized crime dyna...
2      the early life and career of vito corleone in ...
3      when the menace known as the joker emerges fro...
4      a jury holdout attempts to prevent a miscarria...
                             ...                        
245    the desperate life of a chronic alcoholic is f...
246    a something supervising staff member of a resi...
247    a newspaper editor uses every trick in the boo...
248    an old man makes a long journey by lawn mover ...
249    a mumbai teen reflects on his upbringing in th...
Name: clean_plot, Length: 250, dtype: object

In [89]:
# will tokanize sentance into list of words
df['clean_plot'] = df['clean_plot'].apply(lambda x:nltk.word_tokenize(x))
df['clean_plot']

0      [two, imprisoned, men, bond, over, a, number, ...
1      [the, aging, patriarch, of, an, organized, cri...
2      [the, early, life, and, career, of, vito, corl...
3      [when, the, menace, known, as, the, joker, eme...
4      [a, jury, holdout, attempts, to, prevent, a, m...
                             ...                        
245    [the, desperate, life, of, a, chronic, alcohol...
246    [a, something, supervising, staff, member, of,...
247    [a, newspaper, editor, uses, every, trick, in,...
248    [an, old, man, makes, a, long, journey, by, la...
249    [a, mumbai, teen, reflects, on, his, upbringin...
Name: clean_plot, Length: 250, dtype: object

In [90]:
nltk.download('stopwords') # Download stopwords data
stopwords = nltk.corpus.stopwords.words('english') # Correct the language identifier to 'english'
df['clean_plot'] = df['clean_plot'].apply(lambda x:[w for w in x if w not in stopwords])
df['clean_plot']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0      [two, imprisoned, men, bond, number, years, fi...
1      [aging, patriarch, organized, crime, dynasty, ...
2      [early, life, career, vito, corleone, new, yor...
3      [menace, known, joker, emerges, mysterious, pa...
4      [jury, holdout, attempts, prevent, miscarriage...
                             ...                        
245    [desperate, life, chronic, alcoholic, followed...
246    [something, supervising, staff, member, reside...
247    [newspaper, editor, uses, every, trick, book, ...
248    [old, man, makes, long, journey, lawn, mover, ...
249    [mumbai, teen, reflects, upbringing, slums, ac...
Name: clean_plot, Length: 250, dtype: object

In [92]:
df['clean_Writer'] = df['Writer'].str.lower()
df['clean_Writer'] = df['clean_Writer'].astype(str).apply(lambda x:re.sub('[^a-zA-Z]' , ' ', x))
df['clean_Writer'] = df['clean_Writer'].apply(lambda x:re.sub('\s+' , ' ',x))
df['clean_Writer']

0      stephen king short story rita hayworth and sha...
1      mario puzo screenplay francis ford coppola scr...
2      francis ford coppola screenplay mario puzo scr...
3      jonathan nolan screenplay christopher nolan sc...
4          reginald rose story reginald rose screenplay 
                             ...                        
245    charles r jackson from the novel by charles br...
246                                destin daniel cretton
247    charles lederer screen play ben hecht from the...
248                              john roach mary sweeney
249         simon beaufoy screenplay vikas swarup novel 
Name: clean_Writer, Length: 250, dtype: object

In [93]:
df['clean_Writer'] = df['clean_Writer'].apply(lambda x:nltk.word_tokenize(x))
stopwords = nltk.corpus.stopwords.words('english')
df['clean_Writer'] = df['clean_Writer'].apply(lambda x:[w for w in x if w not in stopwords])
df['clean_Writer']

0      [stephen, king, short, story, rita, hayworth, ...
1      [mario, puzo, screenplay, francis, ford, coppo...
2      [francis, ford, coppola, screenplay, mario, pu...
3      [jonathan, nolan, screenplay, christopher, nol...
4      [reginald, rose, story, reginald, rose, screen...
                             ...                        
245    [charles, r, jackson, novel, charles, brackett...
246                            [destin, daniel, cretton]
247    [charles, lederer, screen, play, ben, hecht, p...
248                         [john, roach, mary, sweeney]
249    [simon, beaufoy, screenplay, vikas, swarup, no...
Name: clean_Writer, Length: 250, dtype: object

In [94]:
# split some columns by using comma
df['Director'] = df['Director'].apply(lambda x : x.split(','))
df['Genre'] = df['Genre'].apply(lambda x : x.split(','))
df['Actors'] = df['Actors'].apply(lambda x : x.split(','))
df['Language'] = df['Language'].apply(lambda x : x.split(','))
df['Actors'][0]

['Tim Robbins', ' Morgan Freeman', ' Bob Gunton', ' William Sadler']

In [96]:
# removing space in words
df['Genre'] = df['Genre'].apply(lambda X:[a.lower().replace(' ' , '') for a in X])
df['Actors'] = df['Actors'].apply(lambda X:[a.lower().replace(' ' , '') for a in X])
df['Director'] = df['Director'].apply(lambda X:[a.lower().replace(' ' , '') for a in X])
df['Language'] = df['Language'].apply(lambda X:[a.lower().replace(' ' , '') for a in X])
df['Actors'][0]

['timrobbins', 'morganfreeman', 'bobgunton', 'williamsadler']

In [97]:
# let's combine all sentances again
l=[]
columns = ['Director', 'Genre', 'Actors', 'clean_plot','clean_Writer']
for i in range(len(df)):
  words=''
  for col in columns:
      words+=' '.join(df[col][i])+' '
  l.append(words)


In [98]:
df['clean_input'] = l
df['clean_input']

0      frankdarabont crime drama timrobbins morganfre...
1      francisfordcoppola crime drama marlonbrando al...
2      francisfordcoppola crime drama alpacino robert...
3      christophernolan action crime drama christianb...
4      sidneylumet crime drama martinbalsam johnfiedl...
                             ...                        
245    billywilder drama film-noir raymilland janewym...
246    destindanielcretton drama brielarson johngalla...
247    howardhawks comedy drama romance carygrant ros...
248    davidlynch biography drama sissyspacek janegal...
249    dannyboyle loveleentandan drama devpatel saura...
Name: clean_input, Length: 250, dtype: object

In [99]:
ddf=df[['Title' , 'clean_input']]
ddf.head()

,Title,clean_input
0,The Shawshank Redemption,frankdarabont crime drama timrobbins morganfre...
1,The Godfather,francisfordcoppola crime drama marlonbrando al...
2,The Godfather: Part II,francisfordcoppola crime drama alpacino robert...
3,The Dark Knight,christophernolan action crime drama christianb...
4,12 Angry Men,sidneylumet crime drama martinbalsam johnfiedl...


In [100]:
# feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf = TfidfVectorizer(stop_words='english')
count = CountVectorizer(stop_words='english')
features = tfidf.fit_transform(ddf['clean_input'])
features.shape

(250, 3849)

In [101]:
# calculate Cos Similarity
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(features, features)
cos_sim

array([[1.        , 0.02262611, 0.02224851, ..., 0.0022595 , 0.00265205,
        0.0056169 ],
       [0.02262611, 1.        , 0.4975832 , ..., 0.00228826, 0.00268581,
        0.01593044],
       [0.02224851, 0.4975832 , 1.        , ..., 0.00225007, 0.00264098,
        0.01566458],
       ...,
       [0.0022595 , 0.00228826, 0.00225007, ..., 1.        , 0.00260112,
        0.00248191],
       [0.00265205, 0.00268581, 0.00264098, ..., 0.00260112, 1.        ,
        0.0029131 ],
       [0.0056169 , 0.01593044, 0.01566458, ..., 0.00248191, 0.0029131 ,
        1.        ]])

## Build Recommendation System

In [102]:
# we will use the previous matrix to recommend movies
index = pd.Series(ddf['Title'])
index.head()

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [103]:
# let's build Function
def recommend_movie(title ):
  movies=[]
  idx = index[index==title].index[0]
  score = pd.Series(cos_sim[idx]).sort_values(ascending=False)
  top10 = list(score.iloc[1:11].index)
  for i in top10:
    movies.append(ddf['Title'][i])
  return movies


In [105]:
recommend_movie('The Dark Knight')

['The Dark Knight Rises',
 'Batman Begins',
 'The Prestige',
 'Interstellar',
 'Memento',
 'Inception',
 'The Lion King',
 'Aladdin',
 'Spider-Man: Homecoming',
 'Aliens']